<a href="https://colab.research.google.com/github/AbhiJeet70/GraphPoisoningCodes/blob/main/CaseStudy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary library installations (uncomment when needed)
!pip install torch torch-geometric torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html

# Import libraries
import torch
import torch.nn.functional as F
import random
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.datasets import Planetoid, Amazon, Reddit, Flickr
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx
from torch_geometric.utils import from_networkx
from sklearn.metrics import accuracy_score

# Load the dataset function
def load_dataset(dataset_name):
    if dataset_name in ["Cora", "CiteSeer", "PubMed"]:
        dataset = Planetoid(root=f"./data/{dataset_name}", name=dataset_name)
    elif dataset_name == "Flickr":
        dataset = Flickr(root="./data/Flickr")
    elif dataset_name == "Reddit":
        dataset = Reddit(root="./data/Reddit")
    elif dataset_name == "AmazonProducts":
        dataset = Amazon(root="./data/AmazonProducts", name='Computers')  # 'Computers' or 'Photo'
    else:
        raise ValueError(f"Unsupported dataset: {dataset_name}")

    data = dataset[0]

    # Manually check and set num_classes and num_features if not present
    if not hasattr(data, 'num_classes'):
        data.num_classes = len(data.y.unique())
    if not hasattr(data, 'num_features'):
        data.num_features = data.x.size(1)

    # Convert data to networkx graph (to_undirected)
    G = to_networkx(data, to_undirected=True)
    return data, G

# GCN Model definition
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Evaluation function for GCN
def evaluate_gcn(data, epochs=500):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN(input_dim=data.num_features, hidden_dim=64, output_dim=data.num_classes).to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

    # Train the model
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        pred = model(data).argmax(dim=1)
        correct = pred[data.test_mask] == data.y[data.test_mask]
        accuracy = int(correct.sum()) / int(data.test_mask.sum())

    return accuracy

# Helper function to get a random non-existent edge
def get_random_nonexistent_edge(G):
    while True:
        u = random.choice(list(G.nodes))
        v = random.choice(list(G.nodes))
        if not G.has_edge(u, v) and u != v:
            return u, v

# Poisoning Case 1: Flip percentage of edges from important nodes
def case1_poisoning(G, sorted_node_list, flip_percentage):
    G_poisoned = G.copy()
    num_edges_to_flip = int(flip_percentage * G.number_of_edges())
    flipped_edges = 0

    for node in sorted_node_list:
        if flipped_edges >= num_edges_to_flip:
            break
        node_edges = list(G_poisoned.edges(node))
        num_edges_for_node = int(flip_percentage * len(node_edges))

        for edge in random.sample(node_edges, min(num_edges_for_node, num_edges_to_flip - flipped_edges)):
            G_poisoned.remove_edge(*edge)
            new_edge = get_random_nonexistent_edge(G_poisoned)
            G_poisoned.add_edge(*new_edge)
            flipped_edges += 1

    return G_poisoned

# Poisoning Case 2: Calculate equal number of edges to flip
def case2_poisoning(G, sorted_node_list, flip_percentage):
    G_poisoned = G.copy()
    num_edges_to_flip = int(flip_percentage * G.number_of_edges())
    flipped_edges = 0

    for node in sorted_node_list:
        node_edges = list(G_poisoned.edges(node))
        for edge in node_edges:
            if flipped_edges >= num_edges_to_flip:
                return G_poisoned
            G_poisoned.remove_edge(*edge)
            new_edge = get_random_nonexistent_edge(G_poisoned)
            G_poisoned.add_edge(*new_edge)
            flipped_edges += 1

    return G_poisoned

# Meta poisoning technique
class MLP(torch.nn.Module):
    def __init__(self, input_dim=2, hidden_dim=32, output_dim=1):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Case 3: Meta-poisoning
def case3_meta_poisoning(G, sorted_node_list, flip_percentage, mlp):
    G_poisoned = G.copy()
    num_edges_to_flip = int(flip_percentage * G.number_of_edges())
    flipped_edges = 0
    for node in sorted_node_list:
        if flipped_edges >= num_edges_to_flip:
            break
        node_edges = list(G_poisoned.edges(node))
        for edge in random.sample(node_edges, min(len(node_edges), num_edges_to_flip - flipped_edges)):
            edge_tensor = torch.tensor([G.degree(edge[0]), G.degree(edge[1])], dtype=torch.float32).unsqueeze(0)
            score = mlp(edge_tensor).item()
            if score > 0.5:
                if G_poisoned.has_edge(*edge):
                    G_poisoned.remove_edge(*edge)
                    new_edge = get_random_nonexistent_edge(G_poisoned)
                    G_poisoned.add_edge(*new_edge)
                    flipped_edges += 1

    return G_poisoned

# Case 4: Surrogate GCN + Poisoning
def case4_ugba_with_surrogate(G, sorted_node_list, data):
    G_poisoned = G.copy()

    # Train the surrogate GCN within Case 4
    class SurrogateGCN(torch.nn.Module):
        def __init__(self, input_dim, hidden_dim, output_dim):
            super(SurrogateGCN, self).__init__()
            self.conv1 = GCNConv(input_dim, hidden_dim)
            self.conv2 = GCNConv(hidden_dim, output_dim)
            self.embeddings = None  # Store embeddings after the first layer

        def forward(self, data):
            x, edge_index = data.x, data.edge_index
            x = F.relu(self.conv1(x, edge_index))
            self.embeddings = x  # Store embeddings from the first layer
            x = F.dropout(x, p=0.5, training=self.training)
            x = self.conv2(x, edge_index)
            return F.log_softmax(x, dim=1)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    surrogate_gcn = SurrogateGCN(input_dim=data.num_features, hidden_dim=64, output_dim=data.num_classes).to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(surrogate_gcn.parameters(), lr=0.01, weight_decay=5e-4)

    surrogate_gcn.train()
    for epoch in range(200):  # Train the surrogate GCN
        optimizer.zero_grad()
        out = surrogate_gcn(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

    surrogate_gcn.eval()
    with torch.no_grad():
        embeddings = surrogate_gcn.embeddings.cpu()

    trigger_subgraph = nx.Graph()

    # Create a small homophilic subgraph (trigger) using similar embeddings
    for i in range(len(sorted_node_list)):
        for j in range(i + 1, len(sorted_node_list)):
            if torch.norm(embeddings[sorted_node_list[i]] - embeddings[sorted_node_list[j]]) < 1e-3:
                trigger_subgraph.add_edge(sorted_node_list[i], sorted_node_list[j])

    # Inject the trigger into the poisoned graph
    G_poisoned.add_edges_from(trigger_subgraph.edges())

    return G_poisoned

# Get important nodes based on a given metric
def get_important_nodes(G, metric, order="descending"):
    if metric == "degree":
        node_importance = dict(G.degree())
    elif metric == "eigenvector":
        node_importance = nx.eigenvector_centrality(G)
    elif metric == "closeness":
        node_importance = nx.closeness_centrality(G)
    elif metric == "betweenness":
        node_importance = nx.betweenness_centrality(G)
    elif metric == "katz":
        node_importance = nx.katz_centrality(G)
    elif metric == "pagerank":
        node_importance = nx.pagerank(G)
    else:
        raise ValueError(f"Unknown metric: {metric}")

    return sorted(node_importance, key=node_importance.get, reverse=(order == "descending"))

import matplotlib.pyplot as plt

# Initialize a results dictionary to store accuracies
results = {
    'dataset': [],
    'case_name': [],  # Ensure this key is initialized
    'flip_percentage': [],
    'node_selection_method': [],
    'clean_accuracy': [],
    'poisoned_accuracy': []
}

# Define the datasets, cases, flip percentages, and node selection methods
datasets = ["Cora", "CiteSeer", "PubMed", "Flickr", "Reddit", "AmazonProducts"]
case_names = ['percentile', 'absolute', 'meta', 'ugba']  # Updated to string names
flip_percentages = [0.05, 0.1, 0.15, 0.2, 0.25]
node_selection_methods = ["degree", "eigenvector", "closeness", "betweenness", "katz", "pagerank"]

# Poisoning experiment runner function with node selection by metric
def _experiment(dataset_name, case_name, flip_percentage, node_selection_method, centrality_metric="degree"):
    data, G = load_dataset(dataset_name)

    # Select nodes based on centrality metric
    sorted_node_list = get_important_nodes(G, centrality_metric)

    if case_name == 'percentile':
        G_poisoned = case1_poisoning(G, sorted_node_list, flip_percentage)
    elif case_name == 'absolute':
        G_poisoned = case2_poisoning(G, sorted_node_list, flip_percentage)
    elif case_name == 'meta':
        mlp = MLP()
        G_poisoned = case3_meta_poisoning(G, sorted_node_list, flip_percentage, mlp)
    elif case_name == 'ugba':
        G_poisoned = case4_ugba_with_surrogate(G, sorted_node_list, data)
    else:
        raise ValueError(f"Unsupported case name: {case_name}")

    # Convert the poisoned graph back to PyG format
    data_poisoned = from_networkx(G_poisoned)

    # Copy node features, labels, and masks
    data_poisoned.x = data.x
    data_poisoned.y = data.y
    data_poisoned.train_mask = data.train_mask
    data_poisoned.test_mask = data.test_mask

    # Manually set the num_classes and num_features for poisoned data
    data_poisoned.num_classes = data.num_classes
    data_poisoned.num_features = data.num_features

    # Evaluate GCN performance after poisoning
    clean_accuracy = evaluate_gcn(data, epochs=200)
    poisoned_accuracy = evaluate_gcn(data_poisoned, epochs=200)

    # Store the results
    results['dataset'].append(dataset_name)
    results['case_name'].append(case_name)
    results['flip_percentage'].append(flip_percentage)
    results['node_selection_method'].append(node_selection_method)
    results['clean_accuracy'].append(clean_accuracy)
    results['poisoned_accuracy'].append(poisoned_accuracy)

    print(f"Dataset: {dataset_name}, Case: {case_name}, Flip Percentage: {flip_percentage * 100}%, Node Selection: {node_selection_method}")
    print(f"Clean Accuracy: {clean_accuracy:.4f}, Poisoned Accuracy: {poisoned_accuracy:.4f}")

# Run experiments for each combination
for dataset_name in datasets:
    for case_name in case_names:  # Updated to use case_names list
        for flip_percentage in flip_percentages:
            for node_selection_method in node_selection_methods:
                _experiment(dataset_name=dataset_name, case_name=case_name, flip_percentage=flip_percentage, node_selection_method=node_selection_method)

# Plot results
plt.figure(figsize=(12, 6))

# Clean vs. Poisoned Accuracy
for dataset in set(results['dataset']):
    idx = [j for j, d in enumerate(results['dataset']) if d == dataset]
    plt.plot([results['flip_percentage'][j] for j in idx],
             [results['clean_accuracy'][j] for j in idx],
             marker='o', label=f'{dataset} Clean')

    plt.plot([results['flip_percentage'][j] for j in idx],
             [results['poisoned_accuracy'][j] for j in idx],
             marker='x', linestyle='--', label=f'{dataset} Poisoned')

plt.title('GCN Performance: Clean vs Poisoned Accuracy')
plt.xlabel('Flip Percentage (%)')
plt.ylabel('Accuracy')
plt.xticks([0.05, 0.1, 0.15, 0.2, 0.25])
plt.legend()
plt.grid()
plt.show()



Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Dataset: Cora, Case: percentile, Flip Percentage: 5.0%, Node Selection: degree
Clean Accuracy: 0.8080, Poisoned Accuracy: 0.8060
Dataset: Cora, Case: percentile, Flip Percentage: 5.0%, Node Selection: eigenvector
Clean Accuracy: 0.8070, Poisoned Accuracy: 0.8050
Dataset: Cora, Case: percentile, Flip Percentage: 5.0%, Node Selection: closeness
Clean Accuracy: 0.8100, Poisoned Accuracy: 0.8080
Dataset: Cora, Case: percentile, Flip Percentage: 5.0%, Node Selection: betweenness
Clean Accuracy: 0.8100, Poisoned Accuracy: 0.8000
Dataset: Cora, Case: percentile, Flip Percentage: 5.0%, Node Selection: katz
Clean Accuracy: 0.8070, Poisoned Accuracy: 0.8010
Dataset: Cora, Case: percentile, Flip Percentage: 5.0%, Node Selection: pagerank
Clean Accuracy: 0.8110, Poisoned Accuracy: 0.8060
Dataset: Cora, Case: percentile, Flip Percentage: 10.0%, Node Selection: degree
Clean Accuracy: 0.8040, Poisoned Accuracy: 0.8020
Dataset: Cora, Case